In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler

In [52]:
dados = pd.read_csv('ebay_reviews.csv')
dados = dados.drop(['category'], axis=1)
#Troca-se as notas pela qualidade de negativa ou positiva da review.
dados["rating"]  = dados["rating"].replace([1,2,3,4,5], ["Negativa", "Negativa", "Positiva", "Positiva", "Positiva"])

In [53]:
Vectorizer = CountVectorizer()
Model = LogisticRegression(max_iter = 4000)

#junta-se as informações do título e do conteúdo da review em uma só variável
x = dados['review title'] + " " + dados['review content']
y = dados['rating']
y.value_counts()

Positiva    42814
Negativa     1942
Name: rating, dtype: int64

In [54]:
#Há mais de 42k reviews positivas e apenas 1942 negativas.
#O under sampling balanceia esses dados, pegando 1942 amostras (samples) das 42k positivas.
redutor = RandomUnderSampler()
y = np.array(y)
x = np.array(x)
y = np.reshape(y, (-1,1))
x = np.reshape(x, (-1,1))
x_r, y_r = redutor.fit_resample(x, y)
print(len(y_r))

3884


In [55]:
x_train, x_test, y_train, y_test = train_test_split(x_r,y_r)
x_test = x_test.reshape(-1)
x_train = Vectorizer.fit_transform(x_train.reshape(-1))
x_test = Vectorizer.transform(x_test)
Model.fit(x_train, y_train)
result = Model.predict(x_test)
print(confusion_matrix(y_test, result))
accuracy_do_modelo = accuracy_score(y_test, result)
print(accuracy_do_modelo)

[[414  53]
 [ 59 445]]
0.8846549948506695


In [76]:
import gradio as gr

def funcao_interface(Review):
    negative_prob, positive_prob = Model.predict_proba(Vectorizer.transform([Review]))[0]
    return {"Negativa": negative_prob, "Positiva": positive_prob}

interface = gr.Interface(funcao_interface, gr.inputs.Textbox(placeholder="Escreva sua review aqui"),
gr.outputs.Label(label="Classificação e probabilidades"), live=True, title="Analisador de reviews de produtos",
description="""Escreva uma review genérica para um produto, e o classificador tentará prever se ela é positiva ou negativa.
De preferência, escreva em inglês - uma vez que o programa foi treinado com dados de consumidores majoritariamente anglófonos.""")
interface.launch()

Running on local URL:  http://127.0.0.1:7897/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7897/', None)